In [4]:
import pandas as pd
import nltk
import numpy as np
import re
import random
from nltk.stem import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import pos_tag
from sklearn.metrics import pairwise_distances
from nltk import word_tokenize
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
import speech_recognition as sr
import requests
import pyttsx3
import pyaudio

In [12]:
def net_con():
    url = "http://www.google.com"
    timeout = 4
    try:
        request = requests.get(url, timeout=timeout)
        return True
    except(requests.ConnectionError, requests.Timeout) as exception:
        return False

In [13]:
def speak_output(text2):
    engine = pyttsx3.init()
    engine.say(text2)
    print(text2)
    engine.runAndWait()

In [14]:
def record_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source: 
        print("Listening...")
        audio = r.listen(source)   
        voice_data = " "
        try:
            voice_data = r.recognize_google(audio)
            print(voice_data)
        except:
            print("I did not get that.")
        return voice_data

In [15]:
def text_normalization(text):
    text=str(text).lower()
    special_char_text=re.sub(r'[^a-z]',' ',text)
    tokenized_text=nltk.word_tokenize(special_char_text)
    lemmatized_text=wordnet.WordNetLemmatizer()
    tags_list=pos_tag(tokenized_text,tagset=None)
    lema_words=[]
    for token,pos_token in tags_list:
        if pos_token.startswith('V'):
            pos_val='v'
        elif pos_token.startswith('J'):
            pos_val='a'
        elif pos_token.startswith('R'):
            pos_val='r'
        else:
            pos_val='n'
        lema_token=lemmatized_text.lemmatize(token,pos_val)
        lema_words.append(lema_token)
        
    return " ".join(lema_words)
    

In [16]:
def splitting_of_Input(u_input):
    global b
    q=[]
    a=u_input.split()
    for i in a:
        if i in stop:
            continue
        else:
            q.append(i)
        
        b=" ".join(q)         

In [17]:
Greeting_Inputs = ["hello", "hi", "greetings", "sup", "what's up", "hey", "hai", "hallo"]
Greeting_Responses = ["Hi. I will assist you.", "Hey. I will assist you.", "*nods*", "Hi there. I will assist you.", "Hello. I will assist you.", "I am glad you are talking to me. I will assist you."]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in Greeting_Inputs:
            return random.choice(Greeting_Responses)        

In [18]:
df=pd.read_excel("remibot_dataset_1614271001.xlsx")

In [32]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data] Error downloading 'stopwords' from
[nltk_data]     <https://raw.githubusercontent.com/nltk/nltk_data/gh-
[nltk_data]     pages/packages/corpora/stopwords.zip>:   <urlopen
[nltk_data]     error [SSL: CERTIFICATE_VERIFY_FAILED] certificate
[nltk_data]     verify failed: self-signed certificate (_ssl.c:1006)>


False

In [28]:
df['lemmatized_symptoms']=df['Symptom'].apply(text_normalization)

In [34]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [35]:
stop=stopwords.words('english')
stop_words = ['?',',','.','!','(',')']
for i in stop_words:
    stop.append(i)

In [36]:
def main_f():
    global tfidf
    global df_tfidf
    cv=CountVectorizer()
    x=cv.fit_transform(df['lemmatized_symptoms']).toarray()

    features=cv.get_feature_names()
    df_bow=pd.DataFrame(x,columns=features)

    splitting_of_Input(ques)
        
    ques_lema=text_normalization(b)
    ques_bow=cv.transform([ques_lema]).toarray() 
       
    tfidf=TfidfVectorizer()
    x_tfidf=tfidf.fit_transform(df['lemmatized_symptoms']).toarray()
    
    df_tfidf=pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names())
    ques_tfidf=tfidf.transform([ques_lema]).toarray()

    cosine_value=1-pairwise_distances(df_bow,ques_bow,metric='cosine')
    index_value=cosine_value.argmax()
    if ques != "bye":
        if greeting(ques) != None:
            speak_output(f"Remibot: {greeting(ques)}")
        else:
            for k in chat_tfidf(ques):
                var_n = (f"The Natural Remedies for the following Symptoms are:\n {k}")  
            print("Remibot: ", end=" ")
            speak_output(var_n)

In [37]:
def chat_tfidf(text):
    lemma=text_normalization(text)
    tf=tfidf.transform([lemma]).toarray()
    cos=1-pairwise_distances(df_tfidf,tf,metric='cosine')
    index_value=cos.argmax()
    return [df['Remedies'].loc[index_value]]

In [38]:
def for_offline():
    speak_output("Please type, as speech service is down due to internet connectivity.")
    while continue_diag == True:
        user_input = input("User:")
        ques = user_input.lower()
        if ques == "bye":
            break
        else:
            if ques == "thanks" or ques == "thank you" or ques == "thank you so much" or ques == "thank you very much":
                speak_output("Remibot: Most Welcome")
                break
        main_f()

In [39]:
continue_diag = True
print("", end = " ")
speak_output("Hello I am Remibot! You can talk with me to find Natural Remedies about certain symptoms.")
while True:
    if net_con() == True:
         while continue_diag == True:
            ques = record_audio()
            if ques == "bye":
                break
            else:
                if ques == "thanks" or ques == "thank you" or ques == "thank you so much" or ques == "thank you very much":
                    speak_output("Most Welcome")
                    break
            if net_con == False:
                for_offline()
            main_f()
            if net_con == False:
                for_offline()
    if net_con() == False:
        for_offline()
        
    speak_output("Bye. Take care...")
    break

NameError: name 'pyttsx3' is not defined

In [41]:
import pickle
pickle_out=open('Remibot_N','wb')
pickle.dump(speak_output,pickle_out)
pickle_out.close()

In [42]:
recognizer = sr.Recognizer()

In [43]:
type(recognizer)

speech_recognition.Recognizer